In [ ]:
import cv2
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from options.train_options import TrainOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer

In [ ]:
NAME = 'gen24'

# defaults = {
#     'dataroot': '../../hdr_out/06_Apr_14_16_50',
#     'model': 'hdr',
#     'dataset_mode': 'hdr',
#     'dataset_root': '../../hdr_out/06_Apr_14_16_50',
#     'embedding_save_dir': '',
#     'name': NAME
# }

defaults = {
    'dataroot': '../../generator_data/HFlickr/',
    'model': 'my',
    'dataset_mode': 'my',
    'dataset_root': '../../generator_data/HFlickr/',
    'embedding_save_dir': './checkpoints/emb2',
    'name': NAME
}

# defaults = {
#     'dataroot': '../../embedding_data/vidit/',
#     'model': 'emb',
#     'dataset_mode': 'emb',
#     'dataset_root': '../../embedding_data/vidit/',
# #     'embedding_save_dir': './checkpoints/hdr1/',
#     'name': NAME
# }



    
opt = TrainOptions(defaults=defaults).parse()

In [ ]:
assert opt.isTrain == True

In [ ]:
dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
dataset_size = len(dataset)    # get the number of images in the dataset.
print('The number of training images = %d' % dataset_size)

model = create_model(opt)      # create a model given opt.model and other options
model.setup(opt)               # regular setup: load and print networks; create schedulers
visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
total_iters = 0                # the total number of training iterations

In [ ]:
def to_img(tensor):
    np_img = np.squeeze(tensor.detach().to('cpu').numpy())
    np_img = np_img.transpose((1, 2, 0))
    np_img = cv2.cvtColor(np_img, cv2.COLOR_RGBA2RGB)
    return np_img
    

def display(visuals, epoch):
    real = to_img(visuals['real'])
    comp = to_img(visuals['comp'])
    harmonized = to_img(visuals['harmonized'])
    
    print(f'Epoch {epoch} composition/real/harmonized')
    display = np.concatenate([comp, real, harmonized], axis=1)
    fig = plt.figure(figsize=(15, 15))
    plt.imshow(display)
    plt.show()

def plot_loss_dynamics(X, Y):
    fig = plt.figure(figsize=(8, 8))
    plt.plot(X, Y, label='L1 norm')
    plt.show()

In [ ]:
epoch_losses = []
epochs = []

for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()    # timer for data loading per iteration
    epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch

    epoch_loss = 0.0
    for i, data in enumerate(dataset):  # inner loop within one epoch
        iter_start_time = time.time()  # timer for computation per iteration
        if total_iters % opt.print_freq == 0:
            t_data = iter_start_time - iter_data_time
        visualizer.reset()
        total_iters += opt.batch_size
        epoch_iter += opt.batch_size
        model.set_input(data)         # unpack data from dataset and apply preprocessing
        model.optimize_parameters()   # calculate loss functions, get gradients, update network weights

        if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
            save_result = total_iters % opt.update_html_freq == 0
            model.compute_visuals()
            display(model.get_current_visuals(), epoch)

        if total_iters % opt.print_freq == 0:    
            losses = model.get_current_losses()
            t_comp = (time.time() - iter_start_time) / opt.batch_size
            visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
            epoch_loss += losses['G_L1']

        if total_iters % opt.save_latest_freq == 0:   
            print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
            save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
            model.save_networks(save_suffix)

        iter_data_time = time.time()
    if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
        model.save_networks('latest')
        model.save_networks(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' % 
              (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))
    epoch_losses.append(epoch_loss)
    epochs.append(epoch)
    plot_loss_dynamics(epochs, epoch_losses)
    
    model.update_learning_rate()  